In [8]:
# %env PROJECT_ROOT_PATH=/Users/grubio/Downloads/image/multiple-image-detection/models/research/object_detection/
%env MODEL=ssd_resnet50_v1_fpn_640x640_coco17_tpu-8
%env PIPELINE_CONFIG_FILE=pipeline_ssd_resnet50_v1_fpn_640x640_coco17_tpu-8.config
%env EXPORT_NAME=17-jan-11-40
%env PROJECT_ROOT_PATH=/root/object_detection/object_detection/
# %env MODEL=ssd_resnet50_v1_fpn_640x640_coco17_tpu-8

# %env MODEL=faster_rcnn_inception_resnet_v2_640x640_coco17_tpu-8
# %env PIPELINE_CONFIG_FILE=pipeline_faster_rcnn_inception_resnet_v2_640x640_coco17_tpu-8.config


env: MODEL=ssd_resnet50_v1_fpn_640x640_coco17_tpu-8
env: PIPELINE_CONFIG_FILE=pipeline_ssd_resnet50_v1_fpn_640x640_coco17_tpu-8.config
env: EXPORT_DIR=17-jan-11-40
env: PROJECT_ROOT_PATH=/root/object_detection/object_detection/


In [2]:
from tensorflow.python.client import device_lib

device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 16358096140511168261]

In [11]:
%cd /root/object_detection/object_detection/

/root/object_detection/object_detection


### Train Model

In [14]:
!TF_FORCE_GPU_ALLOW_GROWTH=true python $PROJECT_ROOT_PATH/model_main_tf2.py —logtostderr --model_dir $PROJECT_ROOT_PATH/models/my_$MODEL  --pipeline_config_path $PROJECT_ROOT_PATH/models/my_$MODEL/$PIPELINE_CONFIG_FILE

2021-01-17 12:18:39.283148: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2021-01-17 12:18:42.082863: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-01-17 12:18:42.083531: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2021-01-17 12:18:42.090056: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-01-17 12:18:42.090688: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:05:00.0 name: GeForce RTX 2080 Ti computeCapability: 7.5
coreClock: 1.545GHz coreCount: 68 deviceMemorySize: 10.76GiB deviceMemoryBandwidth: 573.69GiB/s
2021-01-17 12:18:42.090706: I tensorflow/stream_executor/platform/defau

### Export Model

In [15]:
!python $PROJECT_ROOT_PATH/exporter_main_v2.py --input_type image_tensor --pipeline_config_path $PROJECT_ROOT_PATH/models/my_$MODEL/$PIPELINE_CONFIG_FILE --trained_checkpoint_dir $PROJECT_ROOT_PATH/models/my_$MODEL --output_directory $PROJECT_ROOT_PATH/exported-models/$EXPORT_DIR


2021-01-17 15:58:46.533066: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2021-01-17 15:58:49.033729: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-01-17 15:58:49.034378: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2021-01-17 15:58:49.056694: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-01-17 15:58:49.057359: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:05:00.0 name: GeForce RTX 2080 Ti computeCapability: 7.5
coreClock: 1.545GHz coreCount: 68 deviceMemorySize: 10.76GiB deviceMemoryBandwidth: 573.69GiB/s
2021-01-17 15:58:49.057393: I tensorflow/stream_executor/platform/defau

# Evaluate model

In [ ]:
!python $PROJECT_ROOT_PATH/model_main_tf2.py --model_dir $PROJECT_ROOT_PATH/models/my_$MODEL --pipeline_config_path $PROJECT_ROOT_PATH/models/my_$MODEL/$PIPELINE_CONFIG_FILE --checkpoint_dir $PROJECT_ROOT_PATH/models/my_$MODEL


2021-01-17 16:02:35.613497: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
W0117 16:02:38.415942 140105034020672 model_lib_v2.py:1026] Forced number of epochs for all eval validations to be 1.
INFO:tensorflow:Maybe overwriting sample_1_of_n_eval_examples: None
I0117 16:02:38.416068 140105034020672 config_util.py:552] Maybe overwriting sample_1_of_n_eval_examples: None
INFO:tensorflow:Maybe overwriting use_bfloat16: False
I0117 16:02:38.416129 140105034020672 config_util.py:552] Maybe overwriting use_bfloat16: False
INFO:tensorflow:Maybe overwriting eval_num_epochs: 1
I0117 16:02:38.416186 140105034020672 config_util.py:552] Maybe overwriting eval_num_epochs: 1
W0117 16:02:38.416260 140105034020672 model_lib_v2.py:1041] Expected number of evaluation epochs is 1, but instead encountered `eval_on_train_input_config.num_epochs` = 3. Overwriting `num_epochs` to 1.
2021-01-17 16:02:38.423930: I tensorflow/compiler/jit/xla

In [ ]:
!aws s3 sync $PROJECT_ROOT_PATH/models/my_$MODEL s3://detectssss/trainning/$EXPORT_NAME/my_$MODEL
!aws s3 sync $PROJECT_ROOT_PATH/exported-models/ s3://detectssss/trainning/$EXPORT_NAME/exported-models

